# Training Model

In [4]:
import tensorflow as tf
from sklearn.metrics import f1_score
import tensorflow as tf
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Dropout
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import to_categorical
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
  except RuntimeError as e:
    print(e)

In [5]:
import pandas as pd
import numpy as np
import random 

### Importing Data

In [6]:
#Load Data
Data = pd.read_csv("data.csv")

In [7]:
# Convert to DataFrame
df = pd.DataFrame(Data)

In [8]:
# Shuffle Rows
df = df.sample(frac = 1) 

In [9]:
# Convert to np array
dataset = np.array(df)

### Splitting in data and labels

In [10]:
labels = dataset[:, 0] # Labels
data = dataset[:, 1:2501] # Pixel values

### Splitting in Train/Test

In [11]:
#Data Shape
data.shape

(13199, 2500)

In [12]:
#Train
data_train = data[0:10000, :]
data_train = data_train / 255.
#Test
data_test = data[10000:, :]
data_test = data_test / 255.

In [13]:
print(data_train.shape)
print(data_test.shape)

(10000, 2500)
(3199, 2500)


In [14]:
#Labels Shape
labels.shape

(13199,)

In [15]:
#Reshape
labels = labels.reshape(labels.shape[0], 1)
#Train
labels_train = labels[0:10000, :]
labels_train = labels_train.T
#Test
labels_test = labels[10000:, :]
labels_test = labels_test.T

In [16]:
print(labels_train)
print(labels_test)

[[8 4 6 ... 9 6 1]]
[[5 6 1 ... 7 9 3]]


### One-hot Encoding

In [17]:
from tensorflow.keras.utils import to_categorical

In [18]:
y_train = to_categorical(labels_train)
y_test  = to_categorical(labels_test)

### Reshaping variables

In [19]:
train_y = y_train.reshape(y_train.shape[1], y_train.shape[2])
test_y = y_test.reshape(y_test.shape[1], y_test.shape[2])

In [20]:
train_x = data_train.reshape(data_train.shape[0], 50, 50, 1)
test_x = data_test.reshape(data_test.shape[0], 50, 50, 1)

In [21]:
train_x.shape

(10000, 50, 50, 1)

In [22]:
test_x.shape

(3199, 50, 50, 1)

### Model Building

In [23]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Dropout
from keras.callbacks import ModelCheckpoint

In [24]:
# Number of classes
classes = 12

In [25]:
model = Sequential()
model.add(Conv2D(32, (5, 5), input_shape=(50, 50, 1), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))
model.add(Conv2D(64, (5, 5), activation='sigmoid'))
model.add(MaxPooling2D(pool_size=(5, 5), strides=(5, 5), padding='same'))
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.6))
model.add(Dense(classes, activation='softmax'))

c:\Users\nJoy\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


### Compiling Model

In [26]:
# loss=categorical_crossentropy
# optimizer= adam

In [27]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [28]:
# Creating Checkpoint

In [29]:
filepath = "emojinator.keras"
checkpoint1 = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint1]

### Training Model

In [30]:
model.fit(train_x, train_y, validation_data=(test_x, test_y), epochs=25, batch_size=64, callbacks=callbacks_list)

Epoch 1/25
157/157 ━━━━━━━━━━━━━━━━━━━━ 14s 68ms/step - accuracy: 0.5308 - loss: 1.5199 - val_accuracy: 0.9994 - val_loss: 0.0056
Epoch 2/25
  2/157 ━━━━━━━━━━━━━━━━━━━━ 8s 55ms/step - accuracy: 0.9961 - loss: 0.0222  

c:\Users\nJoy\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\callbacks\model_checkpoint.py:206: UserWarning: Can save best model only with val_acc available, skipping.
  self._save_model(epoch=epoch, batch=None, logs=logs)


157/157 ━━━━━━━━━━━━━━━━━━━━ 9s 59ms/step - accuracy: 0.9974 - loss: 0.0128 - val_accuracy: 1.0000 - val_loss: 0.0011
Epoch 3/25
157/157 ━━━━━━━━━━━━━━━━━━━━ 9s 58ms/step - accuracy: 0.9990 - loss: 0.0057 - val_accuracy: 1.0000 - val_loss: 3.0629e-04
Epoch 4/25
157/157 ━━━━━━━━━━━━━━━━━━━━ 9s 58ms/step - accuracy: 0.9999 - loss: 0.0015 - val_accuracy: 1.0000 - val_loss: 3.2828e-04
Epoch 5/25
157/157 ━━━━━━━━━━━━━━━━━━━━ 10s 62ms/step - accuracy: 0.9998 - loss: 0.0017 - val_accuracy: 1.0000 - val_loss: 7.5268e-05
Epoch 6/25
157/157 ━━━━━━━━━━━━━━━━━━━━ 10s 64ms/step - accuracy: 1.0000 - loss: 7.3403e-04 - val_accuracy: 1.0000 - val_loss: 3.6123e-05
Epoch 7/25
157/157 ━━━━━━━━━━━━━━━━━━━━ 10s 64ms/step - accuracy: 1.0000 - loss: 3.3234e-04 - val_accuracy: 1.0000 - val_loss: 2.7011e-05
Epoch 8/25
157/157 ━━━━━━━━━━━━━━━━━━━━ 9s 56ms/step - accuracy: 1.0000 - loss: 2.4613e-04 - val_accuracy: 1.0000 - val_loss: 2.2638e-05
Epoch 9/25
157/157 ━━━━━━━━━━━━━━━━━━━━ 9s 59ms/step - accuracy: 0.99

### Evaluate model on test dataset

In [31]:
scores = model.evaluate(test_x, test_y, verbose=0)

In [32]:
print("Model Accuracy: %.2f%%" % (scores[1] * 100))


Model Accuracy: 99.97%


NameError: name 'history' is not defined

<Figure size 1000x500 with 0 Axes>

### Saving the model

In [ ]:
model.save('emoji_detection.keras')